# 개인 설정

In [ ]:
# 필요 라이브러리 다운로드
!pip install selenium
!pip install bs4
!pip install pandas

In [1]:
# 각자의 데이터 파일의 이름 입력
name = input("데이터 파일의 이름을 입력하시오 :")

데이터 파일의 이름을 입력하시오 :


In [1]:
# 각자의 chromedriver의 위치를 써 넣으시오
driver_path='/Users/sangje/Downloads/chromedriver'

In [14]:
# csv 파일 불러와서 사업자번호 리스트 만들기.
import pandas as pd

data_path=name+'.csv'
df = pd.read_csv(data_path,encoding='cp949') 
# df=pd.read_excel("2908_(송부용) 한국산업단지공단_ESG관련 입주업체 데이터 추출결과_송부용.xlsx") # 원본데이터

num_none=df.isnull().sum().sum()

# 결측치 행 걸러내기 
crawling_col=['기업규모','매출액_2021']

df2=df[(df[crawling_col[0]].isnull()) | (df[crawling_col[1]].isnull())]

query_text=list(df2['업체명'])

print("%s번 크롤링 합니다." % len(query_text))

30044번 크롤링 합니다.


# NICE 기업 정보 크롤링하기 ( 매출액, 기업규모 )

In [15]:
# 필요 라이브러리 import
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import time


# 정보 저장할 리스트
money_list = []
size_list=[]

name = 'Nice_' + name
filename = name + '.csv'  # 파일이름

s = Service(driver_path)
driver = webdriver.Chrome(executable_path=driver_path)
url = 'https://m.nicebizinfo.com/cm/CM0100M001GE.nice'
driver.get(url)
driver.maximize_window()
time.sleep(2)
temp=0

for i in range(len(query_text)):
    temp+=1
    # Nice 로고 누르고 검색어 입력하기
    driver.find_element(By.ID, 'Header_logo').click()
    time.sleep(1)
    
    element=driver.find_element(By.ID, 'CM0100M001GE_itgSrch')
    if str(query_text[i]) != 'nan':
        input_text=query_text[i].replace('-','')
        element.send_keys(input_text)
        element.send_keys("\n")
        time.sleep(1)
    # 검색 결과 데이터가 있으면 클릭하여 크롤링.
    try:
        driver.find_element(By.CLASS_NAME,'company').click()
        time.sleep(1)
        
        driver.execute_script("window.scrollBy(0,6000);")
        time.sleep(1)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        money=soup.find('tbody','aln_c',id='EP0100M002GE_rank').find_all('tr')[1].find('td','aln_r').get_text()
        money=float(money.replace(',',''))
        money_list.append(money)
        
        size=soup.find('div',class_='basicinfo').find('dt',id='EP0100M002GE_eqrObl').get_text().split(',')[0]
        size_list.append(size)
        time.sleep(1)
    # 오류시 출력
    except:
        print("%d번째 크롤링 과정에서 오류가 났습니다." % temp)
        money_list.append(None)
        size_list.append(None)
        time.sleep(1)

# 크롤링 데이터 저장하기
df.loc[df2[:len(money_list)].index,['기업규모']]=size_list
df.loc[df2[:len(money_list)].index,['매출액_2021']]=money_list

num_none2=df.isnull().sum().sum()

print("총 %d 개의 결측치가 해결되었습니다." % num_none-num_none2)

df.to_csv(filename,encoding='cp949')

1번째 크롤링 과정에서 오류가 났습니다.
5번째 크롤링 과정에서 오류가 났습니다.
7번째 크롤링 과정에서 오류가 났습니다.
8번째 크롤링 과정에서 오류가 났습니다.
10번째 크롤링 과정에서 오류가 났습니다.
11번째 크롤링 과정에서 오류가 났습니다.
12번째 크롤링 과정에서 오류가 났습니다.
14번째 크롤링 과정에서 오류가 났습니다.
18번째 크롤링 과정에서 오류가 났습니다.


KeyboardInterrupt: 